In [3]:
import os
import sys
import re
import io
from math import *
import pathlib as pl
import itertools as it
import datetime as dt;
import time;
import random;
import operator as op;
import functools as ft;
import typing as typ;


# Problem 88 - Product-sum numbers

In [4]:
Ints = typ.List[int]

def prod(ns: Ints) -> int:
    accu = 1
    for n in ns:
        accu *= n
    return accu


def psn_delta(ns: Ints) -> int:
    p = prod(ns)
    s = sum(ns)
    return p - s


def is_psn(ns: Ints) -> bool:
    return psn_delta(ns) == 0


In [5]:
def incr_at(ns: Ints, i: int) -> Ints:
    ns[i] += 1
    
    max_i = len(ns) - 1
    i -= 1
    while 0 <= i < max_i and ns[i] < ns[i + 1]:
        ns[i] = ns[i + 1]
        i -= 1
    return ns


assert incr_at([1, 3, 1, 1], 1) == [4, 4, 1, 1]


def min_psn(k: int) -> int:
    ns = [1] * k
    i = 0
    prev_delta = prod(ns) - sum(ns)
    while True:
        incr_at(ns, i)
        p = prod(ns)
        s = sum(ns)
        delta = p - s
        print(f"{ns} i={i} d={delta:>3} {prev_delta:>3} p={p:>3} s={s:>3}")
        if p == s:
            break
        #if delta == prev_delta:
        #    ns[i] = max(1, ns[i] - 1)
        #    delta = prod(ns) - sum(ns)
        #    incr_at(ns, i + 1)
        if p > s or delta == prev_delta:
            ns[i] = max(1, ns[i] - 1)
            delta = prod(ns) - sum(ns)
            i += 1
            
        if delta == 0:
            break

        prev_delta = delta
        
        if i >= k:
            assert False
            
            
    print(ns)
    return sum(ns)
        
print(min_psn(4))
print(min_psn(5))

[2, 1, 1, 1] i=0 d= -3  -3 p=  2 s=  5
[2, 2, 1, 1] i=1 d= -2  -3 p=  4 s=  6
[3, 3, 1, 1] i=1 d=  1  -2 p=  9 s=  8
[3, 2, 2, 1] i=2 d=  4  -1 p= 12 s=  8
[3, 2, 2, 2] i=3 d= 15  -1 p= 24 s=  9


AssertionError: 

In [6]:
def incr_at(ns: Ints, i: int) -> Ints:
    ns[i] += 1
    
    max_i = len(ns) - 1
    i += 1
    while 0 <= i < max_i and ns[i - 1] > ns[i]:
        ns[i] = ns[i - 1]
        i += 1
    print(ns)
    return ns

assert incr_at([1, 3, 1, 1], 1) == [1, 4, 4, 4]

[1, 4, 4, 1]


AssertionError: 

In [7]:
def _min_psn(ns: Ints, i: int) -> int:
    # this is not going to work, because it's depth first
    assert i >= 0
    k = len(ns)
    assert ns[i] <= k
    try:
        ls = _min_psn(list(ns), i - 1)
    except AssertionError:
        ls = None
    
    ns[i] += 1
    p = prod(ns)
    s = sum(ns)
    if p == s:
        if ls is None:
            return s
        else:
            return min(ls, s)
    if ls:
        return ls

In [29]:
def min_psn(k: int, debug: bool = False) -> int:
    ns = [1] * k
    
    p = prod(ns)
    s = sum(ns)
    
    nexts = [(ns, p, s, 0)]
    
    while True:
        cur_min_psn = None
        for ns, p, s, i in nexts:
            if debug:
                print(f"{ns} i={i} p={p:>3} s={s:>3}")
            if p != s:
                continue
            
            print(f"{ns} i={i} p={p:>3} s={s:>3} !!!!!!")
            if cur_min_psn is None:
                cur_min_psn = s
            else:
                cur_min_psn = min(cur_min_psn, s)
                    
        if cur_min_psn:
            return cur_min_psn
        
        new_nexts = []
        for ns, p, s, i in nexts:
            assert p <= s
            if not 0 <= i < k:
                continue
                
            if i == 0 or i > 0 and ns[i - 1] > ns[i]:
                nns = list(ns)
                nns[i] += 1
                p = prod(nns)
                s = sum(nns)
                if p <= s:
                    new_nexts.append((nns, p, s, i))
                    new_nexts.append((nns, p, s, i + 1))
        
        nexts = new_nexts

In [30]:
for i in range(2, 7):
    min_psn(i, debug=True)

[1, 1] i=0 p=  1 s=  2
[2, 1] i=0 p=  2 s=  3
[2, 1] i=1 p=  2 s=  3
[3, 1] i=0 p=  3 s=  4
[3, 1] i=1 p=  3 s=  4
[2, 2] i=1 p=  4 s=  4
[2, 2] i=1 p=  4 s=  4 !!!!!!
[2, 2] i=2 p=  4 s=  4
[2, 2] i=2 p=  4 s=  4 !!!!!!
[1, 1, 1] i=0 p=  1 s=  3
[2, 1, 1] i=0 p=  2 s=  4
[2, 1, 1] i=1 p=  2 s=  4
[3, 1, 1] i=0 p=  3 s=  5
[3, 1, 1] i=1 p=  3 s=  5
[2, 2, 1] i=1 p=  4 s=  5
[2, 2, 1] i=2 p=  4 s=  5
[4, 1, 1] i=0 p=  4 s=  6
[4, 1, 1] i=1 p=  4 s=  6
[3, 2, 1] i=1 p=  6 s=  6
[3, 2, 1] i=1 p=  6 s=  6 !!!!!!
[3, 2, 1] i=2 p=  6 s=  6
[3, 2, 1] i=2 p=  6 s=  6 !!!!!!
[1, 1, 1, 1] i=0 p=  1 s=  4
[2, 1, 1, 1] i=0 p=  2 s=  5
[2, 1, 1, 1] i=1 p=  2 s=  5
[3, 1, 1, 1] i=0 p=  3 s=  6
[3, 1, 1, 1] i=1 p=  3 s=  6
[2, 2, 1, 1] i=1 p=  4 s=  6
[2, 2, 1, 1] i=2 p=  4 s=  6
[4, 1, 1, 1] i=0 p=  4 s=  7
[4, 1, 1, 1] i=1 p=  4 s=  7
[3, 2, 1, 1] i=1 p=  6 s=  7
[3, 2, 1, 1] i=2 p=  6 s=  7
[5, 1, 1, 1] i=0 p=  5 s=  8
[5, 1, 1, 1] i=1 p=  5 s=  8
[4, 2, 1, 1] i=1 p=  8 s=  8
[4, 2, 1, 1] i=1 p=  

In [14]:
# print(sum(set(min_psn(k) for k in range(2, 320))))

13694
